In [3]:
# Setup the Jupyter version of Dash
from jupyter_dash import JupyterDash

# Configure the necessary Python module imports
import dash_leaflet as dl
from dash import dcc
from dash import html
import plotly.express as px
from dash import dash_table
from dash.dependencies import Input, Output
import base64

# Configure OS routines
import os

# Configure the plotting routines
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt



from animal_shelter import AnimalShelter



###########################
# Data Manipulation / Model
###########################

shelter = AnimalShelter()
#We don't need any of the login things since they're hardcoded.  

#USER = 'aacuser'
#PASS = 'aac123'
#HOST = 'nv-desktop-services.apporto.com'
#PORT = 30166 #32067?  
#DB = 'AAC'
#COL = 'animals'

# class read method must support return of list object and accept projection json input
# sending the read method an empty document requests all documents be returned
df = pd.DataFrame.from_records(shelter.read({}))



#########################
# Dashboard Layout / View
#########################
app = JupyterDash('SimpleExample')


#New stuff- Logo, filter names
grazioLogo = base64.b64encode(open('grazio.png', 'rb').read())
rescue_types = ['Water', 'Mountain or Wilderness', 'Disaster or Individual Tracking', 'Reset']


app.layout = html.Div([
    #create the basic dash structure and headers:  
    html.Center(children = [
        html.B(html.H1('NDukes:  Austin Animal Center Dashboard')),
        html.Img(src='data:image/png;base64,{}'.format(grazioLogo.decode()), style={'width': '440px'}),
        html.Center(html.H4("Nate Dukes 2024")),
        html.Div(html.H3("Filters:  "), style={'text-align': 'left'})
    ]),
    html.Hr(),
    
    #create the filters for the rescue types
    dcc.RadioItems(id='rescue-type',
    options=[{'label': types, 'value': types} for types in rescue_types],
    value='Reset', labelStyle={'display': 'block'}),
    html.Hr(),
    
    #create data table
    dash_table.DataTable(
        id='datatable-id',
        columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns],
        data=df.to_dict('records'),
        selected_rows=[0], #default row
        page_size=20, #20 results per page.  Looks much better now.  
    ),
    html.Br(),
    html.Hr(),
    
    
    
    #split the graph and the map- see callback for updateMap()
    html.Div(className='row', style={'display' : 'flex'}, 
        children=[
        html.Div(id='graph-id',className='col s12 m6',),
        html.Div(id='map-id',className='col s12 m6',)]),
    html.Br(),
])

#############################################
# Interaction Between Components / Controller
#############################################
#This callback will highlight a row on the data table when the user selects it
@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')]
)
def update_styles(selected_columns):
    return [{'if': { 'column_id': i },
        'background_color': '#D2F3FF'
    } for i in selected_columns]


@app.callback(Output('datatable-id','data'),
              [Input('rescue-type', 'value')])
def update_dash(filter_type):
    
    #filters are EXACTLY as the pdf describes
    #Are there typos in the database or capitalization differences?  
    #I'm not sure but we'll assume we're working with good data, 
    #and that all the names and filters should display perfectly.  

    if(filter_type == 'Water'):
        return df[df.breed.isin(['Labrador Retriever Mix', 'Chesapeake Bay Retriever', 'Newfoundland'])
                & (df.sex_upon_outcome == 'Intact Female')
                & ((df.age_upon_outcome_in_weeks >= 26)
                & (df.age_upon_outcome_in_weeks <= 156))].to_dict('records')
    
    elif(filter_type == 'Mountain or Wilderness'):
        return df[df.breed.isin(['German Shepherd', 'Alaskan Malamute', 'Old English Sheepdog', 'Siberian Husky', 'Rottweiler'])
                & (df.sex_upon_outcome == 'Intact Male')
                & ((df.age_upon_outcome_in_weeks >= 26) 
                & (df.age_upon_outcome_in_weeks <= 156))].to_dict('records')
    
    elif(filter_type == 'Disaster or Individual Tracking'):
        return  df[df.breed.isin(['Doberman Pinscher', 'German Shepherd', 'Golden Retriever', 'Bloodhound', 'Rottweiler'])
                & (df.sex_upon_outcome == 'Intact Male')
                & ((df.age_upon_outcome_in_weeks >= 20) 
                & (df.age_upon_outcome_in_weeks <= 300))].to_dict('records')
    
    else: #default case
        return df.to_dict('records')

# This callback will update the geo-location chart for the selected data entry
# derived_virtual_data will be the set of data available from the datatable in the form of 
# a dictionary.
# derived_virtual_selected_rows will be the selected row(s) in the table in the form of
# a list. For this application, we are only permitting single row selection so there is only
# one value in the list.
# The iloc method allows for a row, column notation to pull data from the datatable
@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_virtual_data"),
     Input('datatable-id', "derived_virtual_selected_rows")])
def update_map(viewData, index):#I hate python.  a whole week to fix this
    dff = pd.DataFrame.from_dict(viewData) #The map wasn't updating because of indentations
    #If there were scope brackets then my pretty indentation would work fine.  
    
    #add updates based on what row is selected (selected_columns input) in callback?  

    if dff.empty:
        return None
    if index is None or index[0] >= len(dff):
        return None
    
    row = index[0]
    
    return [
        dl.Map(
            style={'width': '1000px', 'height': '500px'}, 
            center=[30.75,-97.48], 
            zoom=10, 
            children=[
                dl.TileLayer(id="base-layer-id"), 
                dl.Marker(
                    position=[dff.iloc[row,13],dff.iloc[row,14]],
                    children=[
                        dl.Tooltip(dff.iloc[row,4]), 
                        dl.Popup([html.H1("Animal Name"), html.P(dff.iloc[row,9])])
                    ]
                )
            ]
        )
    ]


#This graph displays all the breeds (within the filtered set of animals)
#as a pie chart.  It would be pretty easy to change these values
#such that they sort by names, for example, 
#to get data like "most common dog name"
@app.callback(
    Output('graph-id', "children"),
    [Input('datatable-id', "derived_virtual_data")])
def update_graphs(viewData):    
    pieChart = pd.DataFrame.from_dict(viewData)
    if 'breed' in pieChart.columns:
        return [dcc.Graph(figure = px.pie(pieChart, names='breed',))]

app.run_server(debug=False)
#Maybe one day I'll figure out what the stupid "'NoneType' object is not iterable"
#error is caused by.  According to stack overflow I can ignore it though.  


Dash app running on http://127.0.0.1:18700/
